In [1]:
from __future__ import annotations
import os
os.chdir('/home/lq/LQcode/2_project/PHMBench/PHMGA/')
print(f"Current working directory: {os.getcwd()}")
from langgraph.graph import StateGraph, END, START
from src.agents.dataset_preparer_agent import dataset_preparer_agent
from src.agents.execute_agent import execute_agent
from src.agents.inquirer_agent import inquirer_agent
from src.agents.plan_agent import plan_agent
from src.agents.reflect_agent import reflect_agent_node
from src.agents.report_agent import report_agent_node
from src.agents.shallow_ml_agent import shallow_ml_agent
from src.states.phm_states import PHMState
import yaml
import os
from src.utils import load_state, save_state
%load_ext autoreload
%autoreload 2


Current working directory: /home/lq/LQcode/2_project/PHMBench/PHMGA


/home/lq/.conda/envs/PA/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## reload state

In [2]:


config_path = "config/case_exp_ottawa.yaml"
print(f"--- Loading configuration from {config_path} ---")
with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

state_save_path = 'save/exp2.5ottawa/exp2.5_built_state_ottawa.pkl'
builder_cfg = config.get('builder', {})
min_depth = builder_cfg.get('min_depth', 0)
max_depth = builder_cfg.get('max_depth', float('inf'))

# --- Check for existing state ---
if os.path.exists(state_save_path):
    print(f"\n--- Found existing state file at {state_save_path}. Skipping builder workflow. ---")
    state = load_state(state_save_path)






--- Loading configuration from config/case_exp_ottawa.yaml ---

--- Found existing state file at save/exp2.5ottawa/exp2.5_built_state_ottawa.pkl. Skipping builder workflow. ---

--- Loading state from save/exp2.5ottawa/exp2.5_built_state_ottawa.pkl ---
...done.
Successfully loaded state with 23 nodes.


## 重新运行了所以不存在rerun rerun todo

In [ ]:
# from src.utils import initialize_state
# # --- Initialize state if not loaded ---
# ottawa_state_config = "config/case_exp_ottawa.yaml"
# print(f"--- Loading configuration from {ottawa_state_config} ---")
# with open(ottawa_state_config, 'r') as f:
#     ottawa_state_config = yaml.safe_load(f)
# ottawa_state = initialize_state(
#     user_instruction=ottawa_state_config['user_instruction'],
#     metadata_path=ottawa_state_config['metadata_path'],
#     h5_path=ottawa_state_config['h5_path'],
#     ref_ids= ottawa_state_config['ref_ids'],
#     test_ids= ottawa_state_config['test_ids'],
#     case_name="ottawa",
#     use_window= ottawa_state_config.get('use_window', True),)

--- Loading configuration from config/case_exp_ottawa.yaml ---
Loading data for IDs: [47007, 47008, 47009, 47016, 47017, 47018, 47019, 47020, 47021, 47028, 47029, 47030, 47031, 47032, 47033, 47040, 47041, 47042]
Loading data for IDs: [47010, 47011, 47012, 47013, 47014, 47015, 47022, 47023, 47024, 47025, 47026, 47027, 47034, 47035, 47036, 47037, 47038, 47039]


In [ ]:
# state.dag_state.nodes['ch1'] = ottawa_state.dag_state.nodes['ch1']
# state.dag_state.nodes['ch2'] = ottawa_state.dag_state.nodes['ch2']

In [ ]:
# ottawa_state.dag_state.nodes = state.dag_state.nodes.copy()

In [ ]:
# ottawa_state.dag_state.nodes

In [ ]:
# from src.utils.rerun_dag import run_dag_on_new_data

# run_dag_on_new_data(
#     state=ottawa_state)



46it [03:02,  3.96s/it]


## 相似度

In [21]:
# 2. Dataset Preparer Agent: Prepare datasets for ML
print("\nStep 2: Preparing - Creating datasets for ML model...")
preparer_updates = dataset_preparer_agent(state)
state = state.copy(update=preparer_updates)
print(f"Output: Updated state with datasets.")

datasets = preparer_updates['datasets']


Step 2: Preparing - Creating datasets for ML model...


KeyboardInterrupt: 

In [ ]:
print("\nStep 1: Inquiring - Performing similarity analysis...")
inquirer_updates = inquirer_agent(state, metrics=["cosine", "euclidean"])
state = state.copy(update=inquirer_updates)
print(f"Output: Updated state with similarity_results.")

In [7]:
result = shallow_ml_agent(datasets)

--- Ensemble: Found 3 models with CV accuracy > 90% for ensembling.
--- High-quality models: ['wavelet_transform_03_ch1', 'fft_01_hilbert_envelope_01_ch1', 'mean_03_wavelet_transform_03_ch1']


In [10]:
state = state.copy(update={'ml_results': result['metrics_markdown']})

# Save the final state
print(f"\n--- Saving final state to {state_save_path} ---")
save_state(state, state_save_path)

/tmp/ipykernel_644608/3454917170.py:1: PydanticDeprecatedSince20: The `copy` method is deprecated; use `model_copy` instead. See the docstring of `BaseModel.copy` for details about how to handle `include` and `exclude`. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  state = state.copy(update={'ml_results': result['metrics_markdown']})



--- Saving final state to save/exp2.5ottawa/exp2.5_built_state_ottawa.pkl ---

--- Saving state to save/exp2.5ottawa/exp2.5_built_state_ottawa.pkl ---
...done.


True

In [9]:
result_md = result['metrics_markdown']
result_md

'|                                            |   accuracy |       f1 |   cv_accuracy |      cv_f1 |   cv_accuracy_std |   cv_f1_std |\n|:-------------------------------------------|-----------:|---------:|--------------:|-----------:|------------------:|------------:|\n| hilbert_envelope_01_ch1                    |   0.706539 | 0.709033 |      0.758891 |   0.754649 |          0.050594 |    0.055472 |\n| hilbert_envelope_02_ch2                    |   0.339629 | 0.338302 |      0.320853 |   0.319765 |          0.007129 |    0.006871 |\n| wavelet_transform_03_ch1                   |   0.991385 | 0.991385 |      0.988072 |   0.988065 |          0.008678 |    0.008691 |\n| wavelet_transform_04_ch2                   |   0.341507 | 0.338874 |      0.308924 |   0.304770 |          0.007241 |    0.008161 |\n| kurtosis_05_ch1                            |   0.392092 | 0.391570 |      0.407334 |   0.407307 |          0.007945 |    0.008217 |\n| rms_06_ch2                                 |   0.339

## save dataset

In [1]:
# save and load dataset dict
import pickle
import os
save_dir  = '/mnt/crucial/LQ/save/cache'
os.makedirs(save_dir, exist_ok=True)
save_path = os.path.join(save_dir, 'datasets.pkl')
# with open(save_path, 'wb') as f:
#     pickle.dump(datasets, f)
with open(save_path, 'rb') as f:
    datasets = pickle.load(f)

## repotting

In [3]:
# 4. Report Agent: Generate the final report
print("\nStep 4: Reporting - Generating final analysis report...")
report_updates = report_agent_node(state)
state = state.copy(update=report_updates)
print("Output: Updated state with the final report.")


Step 4: Reporting - Generating final analysis report...
Generating final report with 23 nodes, 14 leaves, 2 similarity stats, issues: 0

--- Report Agent LLM Response ---
好的，作为一名PHM（Prognostics and Health Management）领域的报告工程师，我将根据您提供的数据和指令生成最终诊断报告。

***

# 轴承变转速工况故障诊断分析报告

本报告旨在分析所提供的轴承振动信号，以识别潜在的健康状况和故障模式。根据用户指令，本次分析的数据集包含变转速工况下的信号，这意味着故障特征会随转速变化而改变。参考集与测试集均包含三种状态：健康（Health）、内圈故障（Inner Race Fault）和外圈故障（Outer Race Fault）。核心目标是通过将测试信号与参考集进行对比，实现对每个测试信号的精确分类。

## 1. 流程概览

本次诊断任务的处理流程从两个独立的输入通道（`ch1` 和 `ch2`）采集原始振动信号开始。为了从复杂的变转速信号中提取有效的故障特征，我们设计并执行了一个多层次的信号处理与特征提取流程（DAG）：

- **信号预处理**:
  - 对两个通道的原始信号 (`ch1`, `ch2`) 分别应用了 **希尔伯特包络（Hilbert Envelope）** 和 **连续小波变换（Continuous Wavelet Transform）**。希尔ber特包络主要用于解调幅值调制信息，而小波变换则非常适合分析变转速这类非平稳信号的时频特性。

- **特征提取**:
  - **时域与频域统计特征**: 直接从原始信号 `ch1` 和 `ch2` 中提取了多种统计指标，例如峰度（Kurtosis）、峭度因子（Crest Factor）、均方根（RMS）、波形因子（Shape Factor）等。
  - **包络谱特征**: 对希尔伯特包络信号进行 **快速傅里叶变换（FFT）**，得到包络谱。并从包络谱中提取了均值（Mean）、标准差（Std）、峰度（Kurtosis）和偏度（Skew）等特征。
  - **小波特征**: 对小波变换后

/tmp/ipykernel_663415/1608283778.py:4: PydanticDeprecatedSince20: The `copy` method is deprecated; use `model_copy` instead. See the docstring of `BaseModel.copy` for details about how to handle `include` and `exclude`. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  state = state.copy(update=report_updates)


In [4]:
from src.utils import generate_final_report
report = generate_final_report(state,'/mnt/crucial/LQ/save/cache/final_report.md')
print("\nFinal Report:")
print(report)


--- Workflow Finished ---


AttributeError: 'PHMState' object has no attribute 'get'

## plot

In [22]:
from src.utils.visualization import visualize_datasets_umap
save_dir = '/home/lq/LQcode/2_project/PHMBench/PHMGA/save/exp2.5ottawa'
visualize_datasets_umap(datasets, save_dir=save_dir)

--- Generating UMAP visualizations for 21 datasets ---
Processing: hilbert_envelope_01_ch1
  Saved visualization to /home/lq/LQcode/2_project/PHMBench/PHMGA/save/exp2.5ottawa/hilbert_envelope_01_ch1_umap.png
Processing: hilbert_envelope_02_ch2
  Saved visualization to /home/lq/LQcode/2_project/PHMBench/PHMGA/save/exp2.5ottawa/hilbert_envelope_02_ch2_umap.png
Processing: wavelet_transform_03_ch1
  Saved visualization to /home/lq/LQcode/2_project/PHMBench/PHMGA/save/exp2.5ottawa/wavelet_transform_03_ch1_umap.png
Processing: wavelet_transform_04_ch2


: 

: 